In [1]:
# default_exp losses.numpy

In [2]:
#hide
%load_ext autoreload
%autoreload 2

# NumPy Evaluation Metrics

> API details.

In [3]:
#export
from math import sqrt
from typing import Optional, Union

import numpy as np

In [4]:
#export
def divide_no_nan(a, b):
    """
    Auxiliary funtion to handle divide by 0
    """
    div = a / b
    div[div != div] = 0.0
    div[div == float('inf')] = 0.0
    return div

In [5]:
# export
def metric_protections(y: np.ndarray, y_hat: np.ndarray, weights: np.ndarray):
    assert (weights is None) or (np.sum(weights) > 0), 'Sum of weights cannot be 0'
    assert (weights is None) or (weights.shape == y_hat.shape), 'y_hat and weights dimension should be equal'

In [6]:
#export
def mape(y: np.ndarray, y_hat: np.ndarray, 
         weights: Optional[np.ndarray] = None,
         axis: Optional[int] = None) -> Union[float, np.ndarray]:
    """Calculates Mean Absolute Percentage Error.
    
    MAPE measures the relative prediction accuracy of a
    forecasting method by calculating the percentual deviation
    of the prediction and the true value at a given time and
    averages these devations over the length of the series.
    
    Parameters
    ----------
    y: numpy array
        Actual test values.
    y_hat: numpy array
        Predicted values.
    weights: numpy array, optional
        Weights for weighted average.
    axis: None or int, optional
        Axis or axes along which to average a. 
        The default, axis=None, will average over all of the 
        elements of the input array. 
        If axis is negative it counts 
        from the last to the first axis.
 
    Returns
    -------
    mape: numpy array or double
        Return the mape along the specified axis.
    """
    metric_protections(y, y_hat, weights)
        
    delta_y = np.abs(y - y_hat)
    scale = np.abs(y)
    mape = divide_no_nan(delta_y, scale)
    mape = np.average(mape, weights=weights, axis=axis)
    mape = 100 * mape
    
    return mape

In [7]:
#export
def mse(y: np.ndarray, y_hat: np.ndarray, 
        weights: Optional[np.ndarray] = None,
        axis: Optional[int] = None) -> Union[float, np.ndarray]:
    """Calculates Mean Squared Error.
    
    MSE measures the prediction accuracy of a
    forecasting method by calculating the squared deviation
    of the prediction and the true value at a given time and
    averages these devations over the length of the series.
    
    Parameters
    ----------
    y: numpy array
        Actual test values.
    y_hat: numpy array
        Predicted values.
    weights: numpy array
        Weights for weighted average.
    axis: None or int, optional
        Axis or axes along which to average a. 
        The default, axis=None, will average over all of the 
        elements of the input array. 
        If axis is negative it counts 
        from the last to the first axis.
        
    Returns
    -------
    mse: numpy array or double
        Return the mse along the specified axis.
    """
    metric_protections(y, y_hat, weights)

    delta_y = np.square(y - y_hat)
    if weights is not None:
        mse = np.average(delta_y[~np.isnan(delta_y)], 
                         weights=weights[~np.isnan(delta_y)], 
                         axis=axis)
    else:
        mse = np.nanmean(delta_y, axis=axis)
        
    return mse

In [8]:
#export
def rmse(y: np.ndarray, y_hat: np.ndarray,
         weights: Optional[np.ndarray] = None,
         axis: Optional[int] = None) -> Union[float, np.ndarray]:
    """Calculates Root Mean Squared Error.
    
    RMSE measures the prediction accuracy of a
    forecasting method by calculating the squared deviation
    of the prediction and the true value at a given time and
    averages these devations over the length of the series.
    Finally the RMSE will be in the same scale
    as the original time series so its comparison with other
    series is possible only if they share a common scale.
    
    Parameters
    ----------
    y: numpy array
        Actual test values.
    y_hat: numpy array
        Predicted values.    
    weights: numpy array
        Weights for weighted average.
    axis: None or int, optional
        Axis or axes along which to average a. 
        The default, axis=None, will average over all of the 
        elements of the input array. 
        If axis is negative it counts 
        from the last to the first axis.
      
    Returns
    -------
    rmse: numpy array or double
        Return the rmse along the specified axis.
    """

    return np.sqrt(mse(y, y_hat, weights, axis))

In [9]:
#export
def smape(y: np.ndarray, y_hat: np.ndarray,
          weights: Optional[np.ndarray] = None,
          axis: Optional[int] = None) -> Union[float, np.ndarray]:
    """Calculates Symmetric Mean Absolute Percentage Error.
    
    SMAPE measures the relative prediction accuracy of a
    forecasting method by calculating the relative deviation
    of the prediction and the true value scaled by the sum of the
    absolute values for the prediction and true value at a
    given time, then averages these devations over the length
    of the series. This allows the SMAPE to have bounds between
    0% and 200% which is desireble compared to normal MAPE that
    may be undetermined.
    
    Parameters
    ----------
    y: numpy array
        Actual test values.
    y_hat: numpy array
        Predicted values.
    weights: numpy array
        Weights for weighted average.
    axis: None or int, optional
        Axis or axes along which to average a. 
        The default, axis=None, will average over all of the 
        elements of the input array. 
        If axis is negative it counts 
        from the last to the first axis.
    
    Returns
    -------
    smape: numpy array or double
        Return the smape along the specified axis.
    """
    metric_protections(y, y_hat, weights)
        
    delta_y = np.abs(y - y_hat)
    scale = np.abs(y) + np.abs(y_hat)
    smape = divide_no_nan(delta_y, scale)
    smape = 200 * np.average(smape, weights=weights, axis=axis)

    assert all(smape <= 200), 'SMAPE should be lower than 200'
    
    return smape

In [10]:
#export
def mase(y: np.ndarray, y_hat: np.ndarray, 
         y_train: np.ndarray,
         seasonality: int,
         weights: Optional[np.ndarray] = None,
         axis: Optional[int] = None) -> Union[float, np.ndarray]:
    """Calculates the Mean Absolute Scaled Error.

    MASE measures the relative prediction accuracy of a
    forecasting method by comparinng the mean absolute errors
    of the prediction and the true value against the mean
    absolute errors of the seasonal naive model.

    Parameters
    ----------
    y: numpy array
        Actual test values.
    y_hat: numpy array
        Predicted values.
    y_train: numpy array
        Actual insample values for Seasonal Naive predictions.
    seasonality: int
        Main frequency of the time series
        Hourly 24,  Daily 7, Weekly 52,
        Monthly 12, Quarterly 4, Yearly 1.
    weights: numpy array
        Weights for weighted average.
    axis: None or int, optional
        Axis or axes along which to average a. 
        The default, axis=None, will average over all of the 
        elements of the input array. 
        If axis is negative it counts 
        from the last to the first axis.

    Returns
    -------
    mase: numpy array or double
        Return the mase along the specified axis.

    References
    ----------
    [1] https://robjhyndman.com/papers/mase.pdf
    """    
    print("Not implemented yet")

In [11]:
#export
def mae(y: np.ndarray, y_hat: np.ndarray,
        weights: Optional[np.ndarray] = None,
        axis: Optional[int] = None) -> Union[float, np.ndarray]:
    """Calculates Mean Absolute Error.

    The mean absolute error 

    Parameters
    ----------
    y: numpy array
        Actual test values.
    y_hat: numpy array
        Predicted values.
    weights: numpy array
        Weights for weighted average.
    axis: None or int, optional
        Axis or axes along which to average a. 
        The default, axis=None, will average over all of the 
        elements of the input array. 
        If axis is negative it counts 
        from the last to the first axis.

    Returns
    -------
    mae: numpy array or double
        Return the mae along the specified axis.
    """
    metric_protections(y, y_hat, weights)
    
    delta_y = np.abs(y - y_hat)
    if weights is not None:
        mae = np.average(delta_y[~np.isnan(delta_y)], 
                         weights=weights[~np.isnan(delta_y)],
                         axis=axis)
    else:
        mae = np.nanmean(delta_y, axis=axis)
        
    return mae

In [12]:
#export
def pinball_loss(y: np.ndarray, y_hat: np.ndarray, tau: float = 0.5, 
                 weights: Optional[np.ndarray] = None,
                 axis: Optional[int] = None) -> Union[float, np.ndarray]:
    """Calculates the Pinball Loss.

    The Pinball loss measures the deviation of a quantile forecast.
    By weighting the absolute deviation in a non symmetric way, the
    loss pays more attention to under or over estimation.
    A common value for tau is 0.5 for the deviation from the median.

    Parameters
    ----------
    y: numpy array
        Actual test values.
    y_hat: numpy array
        Predicted values.   
    weights: numpy array
        Weights for weighted average.      
    tau: float
        Fixes the quantile against which the predictions are compared.
    axis: None or int, optional
        Axis or axes along which to average a. 
        The default, axis=None, will average over all of the 
        elements of the input array. 
        If axis is negative it counts 
        from the last to the first axis.
    
    Returns
    -------
    pinball loss: numpy array or double
        Return the pinball loss along the specified axis.
    """
    metric_protections(y, y_hat, weights)

    delta_y = y - y_hat
    pinball = np.maximum(tau * delta_y, (tau - 1) * delta_y)

    if weights is not None:
        pinball = np.average(pinball[~np.isnan(pinball)], 
                             weights=weights[~np.isnan(pinball)],
                             axis=axis)
    else:
        pinball = np.nanmean(pinball, axis=axis)
        
    return pinball

In [13]:
# export
def rmae(y: np.ndarray, 
         y_hat1: np.ndarray, y_hat2: np.ndarray, 
         weights: Optional[np.ndarray] = None,
         axis: Optional[int] = None) -> Union[float, np.ndarray]:
    """Calculates Relative Mean Absolute Error.

    The relative mean absolute error of two forecasts.
    A number smaller than one implies that the forecast in the 
    numerator is better than the forecast in the denominator.

    Parameters
    ----------
    y: numpy array
        Actual test values.
    y_hat1: numpy array
        Predicted values of first model.
    y_hat2: numpy array
        Predicted values of second model.
    weights: numpy array
        Weights for weighted average.
    axis: None or int, optional
        Axis or axes along which to average a. 
        The default, axis=None, will average over all of the 
        elements of the input array. 
        If axis is negative it counts 
        from the last to the first axis.

    Returns
    -------
    rmae: numpy array or double
        Return the rmae along the specified axis.
    """
    numerator = mae(y=y, y_hat=y_hat1, weights=weights, axis=axis)
    denominator = mae(y=y, y_hat=y_hat2, weights=weights, axis=axis)
    rmae = numerator / denominator
    
    return rmae

## Multi-quantile NumPy loss

MQLoss definition and testing.

In [14]:
#export
def mqloss(y: np.ndarray, y_hat: np.ndarray, 
           quantiles: np.ndarray, 
           weights: Optional[np.ndarray] = None,
           axis: Optional[int] = None) -> Union[float, np.ndarray]:
    """Calculates the MultiQuantile loss.

    Calculates Average Multi-quantile Loss function, for
    a given set of quantiles, based on the absolute 
    difference between predicted and true values.

    Parameters
    ----------
    y: numpy array
        Actual test values.
    y_hat: numpy array (-1, n_quantiles) 
        Predicted values.
    quantiles: numpy array (n_quantiles) 
        Quantiles to estimate from the distribution of y.
    weights: numpy array
        Weights for weighted average.      
    axis: None or int, optional
        Axis or axes along which to average a. 
        The default, axis=None, will average over all of the 
        elements of the input array. 
        If axis is negative it counts 
        from the last to the first axis.

    Returns
    -------
    mqloss: numpy array or double
        Return the mqloss along the specified axis.
    """ 
    metric_protections(y, y_hat, weights)
    n_q = len(quantiles)
    
    y_rep = np.expand_dims(y, axis=-1)
    error = y_hat - y_rep
    sq = np.maximum(-error, np.zeros_like(error))
    s1_q = np.maximum(error, np.zeros_like(error)) 
    loss = (quantiles * sq + (1 - quantiles) * s1_q)
    loss = np.average(loss, weights=weights, axis=axis)
    
    return loss

In [15]:
y = np.random.random(size=(100, 7))
y_q = np.random.random(size=(100, 7, 4))
weights = np.ones_like(y_q)
quantiles = np.array([0.1, 0.2, 0.3, 0.4])

mqloss(y, y_q, quantiles, weights=weights, axis=(1, 2))

array([0.11319055, 0.1662066 , 0.1687218 , 0.11196051, 0.16984399,
       0.16613338, 0.1082479 , 0.23261391, 0.12315621, 0.20587235,
       0.15831958, 0.17930378, 0.23766872, 0.1736504 , 0.28448128,
       0.14595557, 0.26973386, 0.15280176, 0.12963283, 0.16185545,
       0.15069212, 0.15570959, 0.13827984, 0.15545569, 0.14437847,
       0.17533213, 0.18347565, 0.22509667, 0.1378082 , 0.12803971,
       0.15327662, 0.19277337, 0.14189845, 0.16168525, 0.16889583,
       0.21832227, 0.19470641, 0.14254703, 0.13036127, 0.08885832,
       0.17817867, 0.23140961, 0.19082368, 0.17584046, 0.17888971,
       0.16433528, 0.18602548, 0.13247286, 0.12597067, 0.1352258 ,
       0.12189852, 0.16753081, 0.18609006, 0.12330168, 0.19087797,
       0.21187817, 0.15979956, 0.12609217, 0.16341577, 0.13662247,
       0.11494803, 0.13000434, 0.17797116, 0.21615046, 0.11969575,
       0.18286802, 0.16240748, 0.2069795 , 0.17785966, 0.15366625,
       0.14446143, 0.14480037, 0.18918627, 0.17614856, 0.20385

In [16]:
mqloss(y, y_q, quantiles)

0.16763929502728164

# Checks for NumPy Evaluation Metrics

In [17]:
y = np.array([1,1,1,0,0,0,0,0,1, np.nan])
y_mask = np.array([1,1,1,1,1,1,1,1,2,0])
y_hat = np.array([1,2,3,-4,-5,-6,-7,-8,-9,-10])

print(mae(y=y, y_hat=y_hat, weights=y_mask))
print(mae(y=y, y_hat=y_hat))

5.3
4.777777777777778


In [18]:
print(mae(y=y, y_hat=y_hat, weights=y_mask))
print(mae(y=y, y_hat=y_hat))

5.3
4.777777777777778


In [19]:
len(y)

10